# Heat — single-hazard runner
_Generated on 2025-09-14 15:44 UTC_

## Setup & Config

In [ ]:
# --- Environment & Imports ---
import os, sys, glob, json
from pathlib import Path

# Make sure both the repo root and a potential 'modules/' folder are importable
ROOT = Path.cwd()
if (ROOT / "modules").exists():
    sys.path.insert(0, str(ROOT / "modules"))
sys.path.insert(0, str(ROOT))

# Basic utilities
import matplotlib.pyplot as plt

# YAML config loader (fallback if modules.config_utils not found)
try:
    from modules.config_utils import load_config  # project helper if available
except Exception:
    import yaml
    def load_config(path):
        with open(path, "r", encoding="utf-8") as f:
            return yaml.safe_load(f)

# Helper to auto-detect a config file if not set
DEFAULT_CONFIG_CANDIDATES = ["config.yaml", "config.yml", "configs/config.yaml"]
for _cand in DEFAULT_CONFIG_CANDIDATES:
    if Path(_cand).exists():
        CONFIG_PATH = _cand
        break
else:
    # Put a placeholder if nothing found
    CONFIG_PATH = "config.yaml"  # adjust if needed
print(f"Using CONFIG_PATH = {CONFIG_PATH!r}")
CONFIG = load_config(CONFIG_PATH)
CONFIG

## Run heat processing

In [ ]:
try:
    from modules.heat_module import process_heat, plot_heat_statistics
except Exception:
    from heat_module import process_heat, plot_heat_statistics

heat_raster_path = process_heat(CONFIG)
print("Heat raster (if produced):", heat_raster_path)

# If stats plotting is available (depends on your module implementation), attempt to show
try:
    # Some implementations return stats separately; here we just try to call plot function if present
    # You can adapt this cell to your module's exact API.
    # plot_heat_statistics(annual_stats, CONFIG.get("output_dir", "output"))
    pass
except Exception as e:
    print("[WARN] Heat stats plotting skipped:", e)

In [ ]:
# --- Browse outputs ---
out_dir = Path(CONFIG.get("output_dir", "output"))
out_dir.mkdir(parents=True, exist_ok=True)
maps = sorted(list(out_dir.rglob("*.png"))) + sorted(list(out_dir.rglob("*.jpg")))
rasters = sorted(list(out_dir.rglob("*.tif"))) + sorted(list(out_dir.rglob("*.tiff")))
vectors = sorted(list(out_dir.rglob("*.gpkg"))) + sorted(list(out_dir.rglob("*.geojson")))

print(f"Found {len(maps)} map images, {len(rasters)} rasters, {len(vectors)} vectors under {out_dir}")

# Show the latest map if available
if maps:
    from PIL import Image
    display(Image.open(maps[-1]))
else:
    print("No map images found yet.")